In [2]:
import pandas as pd 

information = pd.read_feather("../../data/datasets/videos/information.feather")

In [3]:
chamber_type, sex, age_group, sample = 3, "male", "young", 0

information.set_index(["chamber", "sex", "age_group", "sample"]).loc[(chamber_type, sex, age_group, sample), ["chronological_age", "ethnicity"]].tolist()

[51, 'Mixed']